In [1]:
import os
import cv2
import albumentations as A
from tqdm import tqdm
import config
import shutil
import random


In [ ]:
## Transforma a resolucao para 256x256
if __name__ == '__main__':
    from pathlib import Path
    from PIL import Image
    # Caminhos
    orig_root = Path("dataset")
    dest_root = Path("dataset-256_v2")

    # Tamanho alvo
    target_size = (256, 256)

    # Extensões válidas
    valid_exts = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}

    # Loop pelas divisões
    for path_conf in [("RGB","train"), ("RGB","val"), 
                  ("gray","annotations/train/semantic_masks"), ("gray","annotations/val/semantic_masks")]:
        
        conf, split = path_conf
        orig_dir = orig_root / split
        dest_dir = dest_root / split

        for root, _, files in os.walk(orig_dir):
            for file in files:
                ext = Path(file).suffix.lower()
                if ext not in valid_exts:
                    continue

                orig_path = Path(root) / file
                rel_path = orig_path.relative_to(orig_root)
                save_path = dest_root / rel_path

                # Cria diretório de destino, se necessário
                save_path.parent.mkdir(parents=True, exist_ok=True)

                # Abre e redimensiona imagem
                try:
                    with Image.open(orig_path) as img:
                        if conf == "RGB":
                            img = img.convert("RGB")  # força RGB
                            img_resized = img.resize(target_size, Image.LANCZOS)
                        else:
                            img = img.convert("P")  # usa palette (1 canal, valores inteiros)
                            img_resized = img.resize(target_size, Image.NEAREST)
                        img_resized.save(save_path)
                except Exception as e:
                    print(f"Erro ao processar {orig_path}: {e}")


In [7]:
# Caminhos
input_image_dir  = config.dataset_path+'train'
input_mask_dir   = config.dataset_path+'annotations/train/semantic_masks'
output_image_dir = config.dataset_path+'train_aug'
output_mask_dir  = config.dataset_path+'annotations/train_aug/semantic_masks'

output_test_image_dir = config.dataset_path+'test'
output_test_mask_dir  = config.dataset_path+'annotations/test/semantic_masks'

In [ ]:
# # MOVE 198 imagens de aleatorias train para test

# Garante que os diretórios de destino existem
os.makedirs(output_test_image_dir, exist_ok=True)
os.makedirs(output_test_mask_dir, exist_ok=True)

# Lista de imagens (assumindo que as máscaras têm o mesmo nome)
image_files = [f for f in os.listdir(input_image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Seleciona 198 aleatoriamente
selected = random.sample(image_files, 198)

# Move arquivos
for fname in selected:
    src_image = os.path.join(input_image_dir, fname)
    src_mask  = os.path.join(input_mask_dir, fname.replace('.jpg','.png'))  # assumindo mesmo nome para a máscara

    dst_image = os.path.join(output_test_image_dir, fname)
    dst_mask  = os.path.join(output_test_mask_dir, fname.replace('.jpg','.png'))

    if os.path.exists(src_image) and os.path.exists(src_mask):
        shutil.move(src_image, dst_image)
        shutil.move(src_mask, dst_mask)
    else:
        print(f"Arquivo ausente: {src_image}")


In [9]:


# Criar diretórios se não existirem
os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_mask_dir, exist_ok=True)

# Número de aumentações por imagem
N = 2

# Transformações
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    #A.VerticalFlip(p=0.5),
    #A.RandomRotate90(p=0.5),
    #A.Rotate(limit=45, p=0.5),
    # A.ShiftScaleRotate(
    #         shift_limit=0.1,
    #         scale_limit=0.1,
    #         rotate_limit=30,
    #         p=0.7,
    #         border_mode=cv2.BORDER_REFLECT
    # ),
    A.RandomBrightnessContrast(p=0.5),
    A.ElasticTransform(p=0.2),
    A.GaussianBlur(p=0.3),
    A.GridDistortion(p=0.2),
    #A.GaussNoise(p=0.2),
])

# Listar imagens
image_filenames = sorted(os.listdir(input_image_dir))

# Aplicar aumentações
for img_name in tqdm(image_filenames):
    img_path  = os.path.join(input_image_dir, img_name)
    mask_path = os.path.join(input_mask_dir, img_name.replace('.jpg','.png'))

    image = cv2.imread(img_path)
    mask  = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

    # Salvar imagem original
    cv2.imwrite(os.path.join(output_image_dir, f"{img_name[:-4]}_orig.jpg"), image)
    cv2.imwrite(os.path.join(output_mask_dir, f"{img_name[:-4]}_orig.png"), mask)

    for i in range(N):
        augmented = transform(image=image, mask=mask)
        aug_img = augmented['image']
        aug_mask = augmented['mask']

        out_img_path = os.path.join(output_image_dir, f"{img_name[:-4]}_aug{i}.jpg")
        out_mask_path = os.path.join(output_mask_dir, f"{img_name[:-4]}_aug{i}.png")

        cv2.imwrite(out_img_path, aug_img)
        cv2.imwrite(out_mask_path, aug_mask)


100%|██████████| 2407/2407 [00:21<00:00, 111.71it/s]
